In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [7]:
#Generates a request for the building to handle.
#For now, I'll assume that the passenger can relay his/her desired destination in the lobby.
#During "downpeak" hours it is likely to get a first floor request going up
#During "uppeak" hours it is likely for a non-first floor request going down to first floor

def generateReq(spec, numFloors):
    if(spec == "uppeak"):
        tmp = np.append([0.5], [0.5 /(numFloors-1)] * (numFloors - 1))
        x = np.random.choice(numFloors, p=tmp)
        y = np.random.choice(numFloors)
        if(x==y):
            return generateReq(spec, numFloors)
        else:
            return x,y
    if(spec == "downpeak"):
        tmp = np.append([0.5], [0.5 /(numFloors-1)] * (numFloors - 1))
        x = np.random.choice(numFloors)
        y = np.random.choice(numFloors, p = tmp)
        if(x==y):
            return generateReq(spec, numFloors)
        else:
            return x,y

In [162]:
#The main class which includes all of the functionality for our system.
class Building():
    def __init__(self, numFloors, numElevators):
        d = {'Floor':[0]*numElevators,
             'Passengers':[0]*numElevators,
             'Capacity':[1]*numElevators,
             'Requests':[None]*numElevators,
             'Direction':['nan']*numElevators,
            }
        self.Elevators = pd.DataFrame(d)
        self.Elevators['Floor'] = np.random.randint(numFloors, size = numElevators)
        e = {'Start':[None],
             'End':[None],
             'STime':[None]
        }
        self.Requests = pd.DataFrame(e)
    
    #Assigns a new request to an elevator. Eventually we want the Neural Network to make this decision.
    def dispatch(self, Request, ID): 
        self.Requests.loc[ID] = Request
        tmp = self.Elevators[self.Elevators['Capacity']!=self.Elevators['Passengers']]
        dists = (tmp['Floor'] - self.Requests.loc[ID,'Start']).abs()
        if(dists.shape[0] == 0):
            print('All elevators are busy')
            return False, 0
        waitTime = np.amin(dists)        
        self.Elevators.loc[np.argmin(dists),'Requests'] = [ID]
        return True, waitTime
        
    #def control(): #Decides how elevators should move. Removes requests if they are finished.
    #Later the Neural Network will also control this decision too.

In [163]:
building = Building(16, 4)
building.Elevators.loc[0,'Passengers'] = 1
building.dispatch([2, 6, 0], 0)
print("New Request: 2nd floor to 6th floor at time t=0")
tmp=building.Elevators.index[building.Elevators['Requests'] == 0].tolist()
print("Request assigned to elevator " + str(tmp))
display(building.Elevators)

New Request: 2nd floor to 6th floor at time t=0
Request assigned to elevator [1]


,Floor,Passengers,Capacity,Requests,Direction
0,14,1,1,None,nan
1,6,0,1,0,nan
2,14,0,1,None,nan
3,15,0,1,None,nan
